#### Monte Carlo para analise de fundo de investimento


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport


In [2]:
data = pd.read_excel('economatica.xlsx', parse_dates=True, index_col=0, skiprows=3) 

In [3]:
data.replace(to_replace=['-'], value=np.nan, inplace=True)
data.dropna(inplace=True)

In [4]:
def columns(df): #renomeando colunas com as ações
    df.columns = df.columns.str[39:]
columns(data)

In [5]:
### Calculando retornos
retornos = data.pct_change().dropna()

In [6]:
eda = ProfileReport(retornos, title='Economatica')


In [7]:
eda.to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
### Contando número de colunas
print(len(retornos.columns))

27


In [ ]:
drawdown = pd.DataFrame()
for i in data.columns:
    roll = pd.DataFrame()
    roll[i] = data[i].rolling(len(data), min_periods=1).max()
    drawdown[i] = data[i]/roll[i] - 1
drawdown.dropna(inplace=True)